In [1]:
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import talib
from talib import MA_Type
import plotly.graph_objects as go
import backtesting
from backtesting import Backtest, Strategy

In [3]:
def LoadAndFill(fileName: str) -> pd.DataFrame:
    df = pd.read_csv(fileName, header=[0, 1], index_col=0)
    df.columns.names = ['Price', 'Date'] # Переименовываем индексное поле из Ticker в Date
    df.index = pd.to_datetime(df.index)
    expected_dates = pd.date_range(start=df.index.min(), end=df.index.max(), freq='D')
    # missing_dates = expected_dates.difference(df.index)
    df = df.reindex(expected_dates)
    df.ffill(inplace=True)
    return df


def ScaleDF(adj_close_df: pd.DataFrame) -> pd.DataFrame:
    scaler = MinMaxScaler()
    scaled_adj_close = pd.DataFrame(
        scaler.fit_transform(adj_close_df),
        columns=adj_close_df.columns,
        index=adj_close_df.index
    )
    return scaled_adj_close


def GetClosePriceForAsset(name: str, input_df: pd.DataFrame) -> pd.DataFrame:  
    input_df = input_df[name].to_frame()
    input_df.rename(columns={name: 'Close'}, inplace=True)
    input_df.rename(columns={'index': 'Date'}, inplace=True)
    return input_df
    
def PlotPrices(df: pd.DataFrame):
    # Plot the scaled data
    df.plot(figsize=(12, 6), title="Нормализованная скорректированная цена закрытия")
    plt.xlabel("Дата")
    plt.ylabel("Цена от 0 до 1")
    plt.grid(True)
    plt.legend(title="Stocks", loc="upper left")
    plt.show()

In [15]:
def bbands_signals(input_df: pd.DataFrame) -> pd.DataFrame:    
    period = 20
    upper, middle, lower = talib.BBANDS(input_df['Close'], matype=MA_Type.SMA, timeperiod=period)
    input_df['upper'], input_df['middle'], input_df['lower'] = upper.astype('float64'), middle.astype('float64'), lower.astype('float64')
    input_df = input_df.iloc[period-1:]
    input_df['Prev Close'] = input_df['Close'].shift(1)

    signal_df = pd.DataFrame(index=input_df.index)
    signal_df['Signal'] = 0


    # Define conditions for buy and sell signals
    buy_condition = (input_df['Prev Close'] >= input_df['lower']) & (input_df['Close'] < input_df['lower'])
    sell_condition = (input_df['Prev Close'] <= input_df['upper']) & (input_df['Close'] > input_df['upper'])
    signal_df.dropna(inplace=True)
    assert signal_df.index.equals(input_df.index), "Indices do not match"
    # Assign signals in 'signal_df'
    # Handle NaNs in conditions
    buy_condition.fillna(False)
    sell_condition.fillna(False)
    signal_df.loc[buy_condition, 'Signal'] = 1   # Buy signal
    signal_df.loc[sell_condition, 'Signal'] = -1  # Sell signal
    signal_df['Close'] = input_df['Close']
    signal_df.reset_index(inplace=True)
    signal_df.rename(columns={'index': 'Date'}, inplace=True)

    buy_sell_signals = signal_df[signal_df['Signal'] != 0]
    buy_sell_signals['prev'] = buy_sell_signals['Signal'].shift(1)

    buy_sell_signals.loc[buy_sell_signals['prev'] == buy_sell_signals['Signal'], 'Signal'] = 0    
    signal_df['Signal'] = buy_sell_signals['Signal']
    signal_df['Signal'].fillna(value=0.0, inplace=True)
    signal_df['Date'] = pd.to_datetime(signal_df['Date']) 

    return signal_df


def macd_signal(input_df: pd.DataFrame) -> pd.DataFrame:    
    # Рассчитываем TEMA и MACD
    signal_df = pd.DataFrame(index=input_df.index)
    signal_df['Close'] = input_df['Close']
    signal_df.reset_index(inplace=True)
    signal_df.rename(columns={'index': 'Date'}, inplace=True)
    signal_df['Signal'] = 0

    signal_df['tema'] = talib.TEMA(signal_df['Close'], timeperiod=24)
    signal_df['macd'], signal_df['macd_signal'], signal_df['macd_hist'] = talib.MACD(signal_df['Close'], fastperiod=12, slowperiod=26, signalperiod=9)
    
    # Создаем сигналы для покупки и продажи
    signal_df['Signal'] = 0
    signal_df.loc[(signal_df['macd'] > signal_df['macd_signal']) & (signal_df['Close'] > signal_df['tema']), 'Signal'] = 1  # Сигнал на покупку
    signal_df.loc[(signal_df['macd'] < signal_df['macd_signal']) & (signal_df['Close'] < signal_df['tema']), 'Signal'] = -1  # Сигнал на продажу
    
    buy_sell_signals = signal_df[signal_df['Signal'] != 0]
    buy_sell_signals['prev'] = buy_sell_signals['Signal'].shift(1)

    buy_sell_signals.loc[buy_sell_signals['prev'] == buy_sell_signals['Signal'], 'Signal'] = 0    
    signal_df['Signal'] = buy_sell_signals['Signal']
    signal_df['Signal'].fillna(value=0.0, inplace=True)
    signal_df['Date'] = pd.to_datetime(signal_df['Date']) 

    return signal_df


def reversed_bbands_signals(input_df: pd.DataFrame) -> pd.DataFrame: 
    signals_df = bbands_signals(input_df)
    signals_df['Signal'] = signals_df['Signal'] * -1
    return signals_df

def reversed_macd_signals(input_df: pd.DataFrame) -> pd.DataFrame: 
    signals_df = macd_signal(input_df)
    signals_df['Signal'] = signals_df['Signal'] * -1
    return signals_df

In [16]:
indticators: list = {
    'BBAND': bbands_signals,
    'MACD': macd_signal,
    'REVERSED_BBAND': reversed_bbands_signals,
    'REVERSED_MACD': reversed_macd_signals,
}

In [5]:
df = LoadAndFill('../hw7/snp500_stock_data.csv')
df = ScaleDF(df['Adj Close'])

In [ ]:
class ComplexStrategy:
    def __init__(self) -> None:
        pass

    def NextDeals(stock_df: pd.DataFrame) -> list:
        for asset_name in stock_df.columns:
            asset_df = GetClosePriceForAsset(asset_name, stock_df)

In [19]:
for indicator in indticators:
    print(indicator)

BBAND
MACD
REVERSED_BBAND
REVERSED_MACD
